In [1]:
!pip install scikit-learn
!pip install imbalanced-learn
!pip install mlxtend
!pip install pandas-profiling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
#from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings("ignore")

In [35]:
df=pd.read_csv('Thyroid_Data.csv')

In [36]:
df.isnull().sum()

Unnamed: 0                      0
age                             1
sex                           110
on_thyroxine                    0
query_on_thyroxine              0
on_antithyroid_medication       0
sick                            0
pregnant                        0
thyroid_surgery                 0
I131_treatment                  0
query_hypothyroid               0
query_hyperthyroid              0
lithium                         0
goitre                          0
tumor                           0
hypopituitary                   0
psych                           0
TSH_measured                    0
TSH                           284
T3_measured                     0
T3                            585
TT4_measured                    0
TT4                           184
T4U_measured                    0
T4U                           297
FTI_measured                    0
FTI                           295
TBG_measured                    0
TBG                          2800
referral_sourc

In [37]:
df=df.drop('TBG',axis=1)

In [38]:
categorical_features=df.select_dtypes(exclude='number')
categorical_features.describe()

,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,query_hyperthyroid,...,hypopituitary,psych,TSH_measured,T3_measured,TT4_measured,T4U_measured,FTI_measured,TBG_measured,referral_source,classes
count,2690,2800,2800,2800,2800,2800,2800,2800,2800,2800,...,2800,2800,2800,2800,2800,2800,2800,2800,2800,2800
unique,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,1,5,4
top,F,f,f,f,f,f,f,f,f,f,...,f,f,t,t,t,t,t,f,other,negative
freq,1830,2470,2760,2766,2690,2759,2761,2752,2637,2627,...,2799,2665,2516,2215,2616,2503,2505,2800,1632,2580


In [39]:
for feature in categorical_features:
    
    print(f"{feature}:{categorical_features[feature].unique()}")

sex:['F' 'M' nan]
on_thyroxine:['f' 't']
query_on_thyroxine:['f' 't']
on_antithyroid_medication:['f' 't']
sick:['f' 't']
pregnant:['f' 't']
thyroid_surgery:['f' 't']
I131_treatment:['f' 't']
query_hypothyroid:['f' 't']
query_hyperthyroid:['f' 't']
lithium:['f' 't']
goitre:['f' 't']
tumor:['f' 't']
hypopituitary:['f' 't']
psych:['f' 't']
TSH_measured:['t' 'f']
T3_measured:['t' 'f']
TT4_measured:['t' 'f']
T4U_measured:['t' 'f']
FTI_measured:['t' 'f']
TBG_measured:['f']
referral_source:['SVHC' 'other' 'SVI' 'STMW' 'SVHD']
classes:['negative' 'compensated hypothyroid' 'primary hypothyroid'
 'secondary hypothyroid']


In [40]:
from sklearn.impute import SimpleImputer

#Handle numerical features
simple_imputer=SimpleImputer(strategy='median')
numerical_missing=pd.DataFrame(simple_imputer.fit_transform(df.select_dtypes(exclude='O')))

#Handle categorical features
cat_imputation=SimpleImputer(strategy='most_frequent')
categorical_missing=pd.DataFrame(cat_imputation.fit_transform(df.select_dtypes(exclude='number')))

numerical_missing.columns=df.select_dtypes(exclude='O').columns
categorical_missing.columns=df.select_dtypes(exclude='number').columns

data=pd.concat([numerical_missing,categorical_missing],axis=1)

In [41]:
data.isnull().sum()

Unnamed: 0                   0
age                          0
TSH                          0
T3                           0
TT4                          0
T4U                          0
FTI                          0
sex                          0
on_thyroxine                 0
query_on_thyroxine           0
on_antithyroid_medication    0
sick                         0
pregnant                     0
thyroid_surgery              0
I131_treatment               0
query_hypothyroid            0
query_hyperthyroid           0
lithium                      0
goitre                       0
tumor                        0
hypopituitary                0
psych                        0
TSH_measured                 0
T3_measured                  0
TT4_measured                 0
T4U_measured                 0
FTI_measured                 0
TBG_measured                 0
referral_source              0
classes                      0
dtype: int64

In [42]:
data[data.age>100]

,Unnamed: 0,age,TSH,T3,TT4,T4U,FTI,sex,on_thyroxine,query_on_thyroxine,...,hypopituitary,psych,TSH_measured,T3_measured,TT4_measured,T4U_measured,FTI_measured,TBG_measured,referral_source,classes
1364,1364.0,455.0,1.1,2.0,118.0,1.13,104.0,F,f,f,...,f,f,t,t,t,t,t,f,SVI,negative


In [43]:
data=data.drop(data.age.index[1364])

In [44]:
import numpy as np
def outliers_removal(numerical_missing):
    for column in numerical_missing:
        sort=np.sort(numerical_missing[column])
        lower_limit,upper_limit=np.percentile(sort,[0,95])
        detected_outliers=numerical_missing.iloc[np.where((numerical_missing[column]>upper_limit) | (numerical_missing[column]<lower_limit))]
        return detected_outliers


In [45]:
outliers_data=outliers_removal(data)
outliers_data

,Unnamed: 0,age,TSH,T3,TT4,T4U,FTI,sex,on_thyroxine,query_on_thyroxine,...,hypopituitary,psych,TSH_measured,T3_measured,TT4_measured,T4U_measured,FTI_measured,TBG_measured,referral_source,classes
2660,2660.0,31.0,3.10,1.8,120.0,1.22,99.0,F,f,f,...,f,f,t,t,t,t,t,f,other,negative
2661,2661.0,65.0,0.40,2.0,175.0,1.12,156.0,F,t,f,...,f,f,t,f,t,t,t,f,other,negative
2662,2662.0,83.0,19.00,2.0,148.0,1.11,134.0,F,f,f,...,f,f,t,f,t,t,t,f,other,compensated hypothyroid
2663,2663.0,19.0,3.70,1.1,126.0,0.91,138.0,F,f,f,...,f,f,t,t,t,t,t,f,SVI,negative
2664,2664.0,75.0,0.97,0.6,80.0,0.72,110.0,M,f,f,...,f,f,t,t,t,t,t,f,SVI,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,2795.0,70.0,2.70,2.0,155.0,1.05,148.0,M,f,f,...,f,f,t,f,t,t,t,f,SVI,negative
2796,2796.0,73.0,1.40,0.7,63.0,0.88,72.0,M,f,t,...,f,f,f,t,t,t,t,f,other,negative
2797,2797.0,75.0,1.40,2.0,147.0,0.80,183.0,M,f,f,...,f,f,f,f,t,t,t,f,other,negative
2798,2798.0,60.0,1.40,2.0,100.0,0.83,121.0,F,f,f,...,f,f,t,f,t,t,t,f,other,negative


In [46]:
new_df=data.drop(outliers_data.index)
new_df.head()

,Unnamed: 0,age,TSH,T3,TT4,T4U,FTI,sex,on_thyroxine,query_on_thyroxine,...,hypopituitary,psych,TSH_measured,T3_measured,TT4_measured,T4U_measured,FTI_measured,TBG_measured,referral_source,classes
0,0.0,41.0,1.30,2.5,125.0,1.14,109.0,F,f,f,...,f,f,t,t,t,t,t,f,SVHC,negative
1,1.0,23.0,4.10,2.0,102.0,0.98,107.0,F,f,f,...,f,f,t,t,t,f,f,f,other,negative
2,2.0,46.0,0.98,2.0,109.0,0.91,120.0,M,f,f,...,f,f,t,f,t,t,t,f,other,negative
3,3.0,70.0,0.16,1.9,175.0,0.98,107.0,F,t,f,...,f,f,t,t,t,f,f,f,other,negative
4,4.0,70.0,0.72,1.2,61.0,0.87,70.0,F,f,f,...,f,f,t,t,t,t,t,f,SVI,negative


In [47]:
new_df.shape

(2659, 30)

In [48]:
new_df.columns

Index(['Unnamed: 0', 'age', 'TSH', 'T3', 'TT4', 'T4U', 'FTI', 'sex',
       'on_thyroxine', 'query_on_thyroxine', 'on_antithyroid_medication',
       'sick', 'pregnant', 'thyroid_surgery', 'I131_treatment',
       'query_hypothyroid', 'query_hyperthyroid', 'lithium', 'goitre', 'tumor',
       'hypopituitary', 'psych', 'TSH_measured', 'T3_measured', 'TT4_measured',
       'T4U_measured', 'FTI_measured', 'TBG_measured', 'referral_source',
       'classes'],
      dtype='object')

In [49]:
new_df.to_csv('Preprocessed_data.csv',index=False)